# VacationPy
----

In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [ ]:
# Path to the csv file
cities_csv = "../output_data/cities.csv"

# Ready the weather data for all cities into a DataFrame
cities_weather_df = pd.read_csv(cities_csv)

# Print DataFrame
cities_weather_df.head()

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [ ]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Grab the latitude and longitude locations and humidity data 
locations = cities_weather_df[['Latitude', 'Longitude']]
humidity = cities_weather_df['Humidity'].astype(float)
max_humidity = humidity.max()

In [ ]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer by using Lat and Lng as location and humidity as the weight
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=float(max_humidity),
                                 point_radius=1)

# Add heatmap layer to map
fig.add_layer(heat_layer)

# Display figure
fig

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [ ]:
# Narrow down the DataFrame based on ideal weather conditions to return an ideal cities DataFrame
# A max temperature less than or equal to 80 degrees but higher than or equal to 65
# Wind speed less than or equal to 10 mph
# Not more than 30% cloud cover
# Not more than 80% humdity

ideal_weather_conditions_df = cities_weather_df[ (cities_weather_df['Max Temperature'] >= 65) & (cities_weather_df['Max Temperature'] <= 80) & (cities_weather_df['Wind Speed'] <= 10) & (cities_weather_df['Cloudiness'] <= 30)  & (cities_weather_df['Humidity'] <= 80)]

# Drop any rows with null values
ideal_cities_df = ideal_weather_conditions_df.dropna(how='any')
ideal_cities_df.count()

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
# Set Base URL for the Google Places API
places_api_url = f'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

# Set up lists to hold hotel names, their geo-coordinates, city and country 
hotels_list = []
hotels_lat = []
hotels_lng = []
cities = []
countries = []

# Start of print log as we begin data retrieval using the API
print("Searching for Hotels...")
print(f"-" * 90)

# Iterate over the ideal cities DataFrame to find the first hotel for each city located within 5000 meters of the geo-coordinates.
for index, row in ideal_cities_df.iterrows(): 
    # Setup a dictionary for the parameters to do a 'nearbysearch' of hotels using the Google Places API
    # Parameters include:
    # - the lat/long of city around which to retrieve place information
    # - the radius (in meters) within which to return place results
    # - type of place, i.e. lodging/hotels that we want to search for
    params = { 'location': f"{row['Latitude']}, {row['Longitude']}", 'radius': 5000, 'type': 'lodging', 'key': g_key }
    
    # Execute the Google Place API call with the defined parameters and store the results into a JSON response object
    places_response = requests.get(places_api_url, params=params).json()

    # Try to grab the Hotel Name and it's Geo-Coordinates if available
    try:
        hotel_name = places_response['results'][0]['name']
        hotels_list.append(hotel_name)
        hotels_lat.append(places_response['results'][0]['geometry']['location']['lat'])
        hotels_lng.append(places_response['results'][0]['geometry']['location']['lng'])
        cities.append(row['City'])
        countries.append(row['Country'])
        # Print a log of each city as it's being processed with the city number and city name=
        print(f"Found the hotel '{hotel_name}' within 5000 meters of the city '{row['City']}'")

    # Handle exceptions for any hotels that could not be found within the 5000 meter radius 
    except:
        print(f"Cound not find any hotel within 5000 meters of the city '{row['City']}' | Skipping to the next one...")
        pass

# Print the final log indicating that the search has been completed
print("-" * 90)
print(f"Completed Hotel Search | Below is a list of all {len(hotels_list)} Hotels...")      
print("-" * 90)

# Save the Hotel results to a DataFrame and print it
hotel_df = pd.DataFrame( {'Hotel Name': hotels_list, 'Hotel Lat': hotels_lat, 'Hotel Lng': hotels_lng, 'City': cities, 'Country': countries} )
hotel_df

In [ ]:
# Using the template add the hotel markers to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# List of hotel marker locations represents the lat/long coordinates of each hotel
hotel_marker_locations = hotel_df[["Hotel Lat", "Hotel Lng"]]

In [ ]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(hotel_marker_locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display Map
fig